# Evaluation Metrics: SCOT

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv
import solaris.eval.scot as scot

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
ground_truth = pd.read_csv('/local_storage/users/hfang/inference_winner_split/sn7_baseline_preds/csvs/ground_truth.csv')
aois = [z.split('mosaic_')[-1] for z in ground_truth['filename'].values]
ground_truth['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in ground_truth['filename'].values]
ground_truth['timestep'] = timesteps
ground_truth = ground_truth.drop('filename', 1)

In [3]:
ground_truth.head()

,id,geometry,aoi,timestep
0,1131,"POLYGON ((1022.739556749351 1023.915693833842,...",L15-0387E-1276N_1549_3087_13,2018_01
1,896,"POLYGON ((839.4174400521442 1023.860588716227,...",L15-0387E-1276N_1549_3087_13,2018_01
2,607,"POLYGON ((977.5393254966475 1021.220368818729,...",L15-0387E-1276N_1549_3087_13,2018_01
3,539,"POLYGON ((804.8580670831725 1021.935737615102,...",L15-0387E-1276N_1549_3087_13,2018_01
4,769,"POLYGON ((817.6027463246137 1022.111423592898,...",L15-0387E-1276N_1549_3087_13,2018_01


In [4]:
ground_truth['geometry'] = ground_truth['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(ground_truth, geometry = 'geometry')
print(gdf)

          id                                           geometry  \
0       1131  POLYGON ((1022.740 1023.916, 1021.780 1025.799...   
1        896  POLYGON ((839.417 1023.861, 832.526 1024.057, ...   
2        607  POLYGON ((977.539 1021.220, 976.830 1024.765, ...   
3        539  POLYGON ((804.858 1021.936, 801.562 1024.604, ...   
4        769  POLYGON ((817.603 1022.111, 814.501 1024.105, ...   
...      ...                                                ...   
953426   199  POLYGON ((16.975 27.234, 23.288 27.566, 23.158...   
953427   215  POLYGON ((264.036 7.059, 267.359 7.059, 267.35...   
953428   225  POLYGON ((269.021 3.875, 273.175 3.875, 273.17...   
953429   648  POLYGON ((267.221 3.459, 267.221 1.659, 269.15...   
953430   625  POLYGON ((268.467 0.829, 268.467 -0.833, 271.7...   

                                 aoi timestep  
0       L15-0387E-1276N_1549_3087_13  2018_01  
1       L15-0387E-1276N_1549_3087_13  2018_01  
2       L15-0387E-1276N_1549_3087_13  2018_01  
3  

In [5]:
pred = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7/submission_hrnet.csv')
aois = [z.split('mosaic_')[-1] for z in pred['filename'].values]
pred['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in pred['filename'].values]
pred['timestep'] = timesteps
pred = pred.drop('filename', 1)

In [6]:
pred.head()

,id,geometry,aoi,timestep
0,0,"POLYGON (( 2.7 37,2.7 37.3,2 37.7,2 39,1.7 39,...",L15-0632E-0892N_2528_4620_13,2018_01
1,0,"POLYGON (( 2.7 37,2.7 37.3,2 37.7,2 39,1.7 39,...",L15-0632E-0892N_2528_4620_13,2018_02
2,0,"POLYGON (( 2.7 37,2.7 37.3,2 37.7,2 39,1.7 39,...",L15-0632E-0892N_2528_4620_13,2018_03
3,0,"POLYGON (( 2.7 37,2.7 37.3,2 37.7,2 39,1.7 39,...",L15-0632E-0892N_2528_4620_13,2018_04
4,0,"POLYGON (( 2.7 37,2.7 37.3,2 37.7,2 39,1.7 39,...",L15-0632E-0892N_2528_4620_13,2018_05


In [7]:
pred['geometry'] = pred['geometry'].apply(wkt.loads)
pdf = gpd.GeoDataFrame(pred, geometry = 'geometry')
print(pdf)

          id                                           geometry  \
0          0  POLYGON ((2.700 37.000, 2.700 37.300, 2.000 37...   
1          0  POLYGON ((2.700 37.000, 2.700 37.300, 2.000 37...   
2          0  POLYGON ((2.700 37.000, 2.700 37.300, 2.000 37...   
3          0  POLYGON ((2.700 37.000, 2.700 37.300, 2.000 37...   
4          0  POLYGON ((2.700 37.000, 2.700 37.300, 2.000 37...   
...      ...                                                ...   
485743  2105  POLYGON ((941.000 1022.300, 941.000 1022.700, ...   
485744  2105  POLYGON ((941.000 1022.300, 941.000 1022.700, ...   
485745  2105  POLYGON ((941.000 1022.300, 941.000 1022.700, ...   
485746  2105  POLYGON ((941.000 1022.300, 941.000 1022.700, ...   
485747  2105  POLYGON ((941.000 1022.300, 941.000 1022.700, ...   

                                 aoi timestep  
0       L15-0632E-0892N_2528_4620_13  2018_01  
1       L15-0632E-0892N_2528_4620_13  2018_02  
2       L15-0632E-0892N_2528_4620_13  2018_03  
3  

In [8]:
scot.scot_multi_aoi(gdf, pdf, verbose=True)


1 / 10: AOI L15-0387E-1276N_1549_3087_13
Matching footprints
   1: F1 = 0.2811
   2: F1 = 0.2844
   3: F1 = 0.2796
   4: F1 = 0.2811
   5: F1 = 0.2792
   6: F1 = 0.2791
   7: F1 = 0.2785
   8: F1 = 0.2784
   9: F1 = 0.2795
  10: F1 = 0.2798
  11: F1 = 0.2895
  12: F1 = 0.2845
  13: F1 = 0.2806
  14: F1 = 0.2812
  15: F1 = 0.2809
  16: F1 = 0.2821
  17: F1 = 0.2834
  18: F1 = 0.2847
  19: F1 = 0.2857
  20: F1 = 0.2931
  21: F1 = 0.2907
  22: F1 = 0.2808
Identifying mismatches
Tracking:
    Mismatches: 4
      True Pos: 7389
     False Pos: 7837
     False Neg: 29696
   Track Score: 0.2825
Change Detection:
      True Pos: 68
     False Pos: 366
     False Neg: 439
  Change Score: 0.1445
Combined Score: 0.2372

2 / 10: AOI L15-0566E-1185N_2265_3451_13
Matching footprints
   1: F1 = 0.3226
   2: F1 = 0.2812
   3: F1 = 0.2769
   4: F1 = 0.2535
   5: F1 = 0.3200
   6: F1 = 0.3500
   7: F1 = 0.3448
   8: F1 = 0.2880
   9: F1 = 0.2969
  10: F1 = 0.3134
  11: F1 = 0.2901
  12: F1 = 0.3104
  1

(0.22865491421756196,
 {'L15-0387E-1276N_1549_3087_13': [4,
   7389,
   7837,
   29696,
   0.2825027240924471,
   68,
   366,
   439,
   0.14452709883103082,
   0.23721108293734183],
  'L15-0566E-1185N_2265_3451_13': [10,
   1173,
   1400,
   4016,
   0.3022416902860088,
   63,
   218,
   596,
   0.13404255319148936,
   0.2416070650915896],
  'L15-0632E-0892N_2528_4620_13': [1,
   14253,
   28066,
   49587,
   0.26852174568336173,
   34,
   179,
   99,
   0.19653179190751446,
   0.2501925483405013],
  'L15-1015E-1062N_4061_3941_13': [6,
   22056,
   24889,
   34774,
   0.42507347627077813,
   137,
   855,
   541,
   0.16407185628742516,
   0.32247603098411115],
  'L15-1200E-0847N_4802_4803_13': [0,
   41634,
   63363,
   224680,
   0.22425406195884312,
   25,
   135,
   165,
   0.14285714285714285,
   0.20131327002301608],
  'L15-1276E-1107N_5105_3761_13': [2,
   28105,
   18773,
   45045,
   0.4683073949411804,
   78,
   637,
   639,
   0.10893854748603352,
   0.2821529430892266],
  '